In [1]:
import networkx as nx
import csv
import numpy as np
import pandas as pd
import os
from collections import defaultdict
from node2vec import Node2Vec
from gensim.models import Word2Vec

import keras
import tensorflow as tf
from keras.layers import Dense, Dropout,Reshape, Activation
from keras.models import Sequential
from keras.optimizers import SGD
from keras.layers.merge import Concatenate

from sklearn.model_selection import KFold 

Using TensorFlow backend.


In [2]:
                                                # SETTINGS
nb_emb_w2v = 30
nb_topics = 24
w2vModel = Word2Vec.load("Files/word2vec_30.model")

In [3]:
                        # FORMAT TRAINING : SEPARATE GRAPH INFORMATION FROM TEXT INFORMATION

nlpDataframe2 = pd.read_csv("Files/nlpTrain.csv")

Link = nlpDataframe2.pop('Link')
CosSimilarityDf = nlpDataframe2[['Cos_similarity']]
nlpDataframe2.pop('Cos_similarity')

topic_list = []

for i in range(nb_emb_w2v):
    topic_name = "1_Emb "+ str(i)
    topic_list.append(topic_name)

for i in range(nb_emb_w2v):
    topic_name = "2_Emb "+ str(i)
    topic_list.append(topic_name)
    

embDf = pd.DataFrame(0.0, index=np.arange(nlpDataframe2.shape[0]), columns= topic_list)


for index,row in nlpDataframe2.iterrows():
    #print(index)
    if (index == 50000):
        print (50000)
        
    idx1 = int(row["Node1"])
    idx2 = int(row["Node2"])
    
    try :
        node1_values = w2vModel[str(idx1)]
    except KeyError:
        node1_values = [0.0 for i in range(nb_emb_w2v)]
    
    try :
        node2_values = w2vModel[str(idx2)]
    except KeyError:
        node2_values = [0.0 for i in range(nb_emb_w2v)]

    for i,(v1,v2) in enumerate(zip(node1_values, node2_values)):
        
        n1_emb_name = "1_Emb "+ str(i)
        n2_emb_name = "2_Emb "+ str(i)
        
        embDf.at[index, n1_emb_name] = v1
        embDf.at[index, n2_emb_name] = v2

pop1 = nlpDataframe2.pop("Node1")
pop2 = nlpDataframe2.pop("Node2")

/usr/local/Cellar/ipython/7.8.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:32: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/usr/local/Cellar/ipython/7.8.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:37: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


50000


In [4]:
nlpDataframe2.head()

,1_Topic 0,1_Topic 1,1_Topic 2,1_Topic 3,1_Topic 4,1_Topic 5,1_Topic 6,1_Topic 7,1_Topic 8,1_Topic 9,...,2_Topic 14,2_Topic 15,2_Topic 16,2_Topic 17,2_Topic 18,2_Topic 19,2_Topic 20,2_Topic 21,2_Topic 22,2_Topic 23
0,0.0,0.0,0.0,0.092979,0.0,0.398997,0.000000,0.0,0.0,0.0,...,0.020837,0.020837,0.020837,0.020837,0.020837,0.520757,0.020837,0.020837,0.020837,0.020837
1,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.959254,0.000000,0.000000,0.000000,0.000000,0.040073,0.000000
2,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.371359,0.000000,0.000000,0.000000
3,0.0,0.0,0.0,0.000000,0.0,0.000000,0.987991,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.134329,0.865167,0.000000,0.000000,0.000000,0.000000
4,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [5]:
                                    # K FOLD MODEL TRAINING FOR NEURAL NET
listModel = []

kf = KFold(n_splits=10, random_state=42, shuffle=True)# Define the split - into 10 folds 

for train_index, test_index in kf.split(embDf):
    
    X_train, X_test = nlpDataframe2.loc[train_index], nlpDataframe2.loc[test_index]
    y_train, y_test = Link[train_index], Link[test_index]
    model = Sequential()
    model.add(Dense(60, activation='relu', input_dim = nb_topics*2))
    model.add(Dense(60, activation='relu'))
    model.add(Dense(30, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

    model.fit(X_train, y_train, epochs=6, batch_size=32)
    listModel.append(model)

Epoch 1/6
408417/408417 [==============================] - 23s 55us/step - loss: 0.5433 - accuracy: 0.7199
Epoch 2/6
408417/408417 [==============================] - 23s 56us/step - loss: 0.5207 - accuracy: 0.7385
Epoch 3/6
408417/408417 [==============================] - 23s 57us/step - loss: 0.5157 - accuracy: 0.7428
Epoch 4/6
408417/408417 [==============================] - 25s 61us/step - loss: 0.5128 - accuracy: 0.7455
Epoch 5/6
408417/408417 [==============================] - 25s 60us/step - loss: 0.5117 - accuracy: 0.7467
Epoch 6/6
408417/408417 [==============================] - 23s 56us/step - loss: 0.5109 - accuracy: 0.7474
Epoch 1/6
408417/408417 [==============================] - 23s 56us/step - loss: 0.5432 - accuracy: 0.7199
Epoch 2/6
408417/408417 [==============================] - 23s 57us/step - loss: 0.5191 - accuracy: 0.7386
Epoch 3/6
408417/408417 [==============================] - 24s 59us/step - loss: 0.5134 - accuracy: 0.7431
Epoch 4/6
408417/408417 [============

In [6]:
# choose the index of the best performing net
index = 1
model = listModel[index]

In [7]:
                                           # FORMAT SUBMISSION DATASET

    
nlpTestDataframe = pd.read_csv('Files/nlpTest.csv')
CosSimilarityTestDf = nlpTestDataframe[['Cos_similarity']]
nlpTestDataframe.pop('Cos_similarity')

topic_list = []

for i in range(nb_emb_w2v):
    topic_name = "1_Emb "+ str(i)
    topic_list.append(topic_name)

for i in range(nb_emb_w2v):
    topic_name = "2_Emb "+ str(i)
    topic_list.append(topic_name)
    

embDfTest = pd.DataFrame(0.0, index=np.arange(nlpTestDataframe.shape[0]), columns= topic_list)

for index,row in nlpTestDataframe.iterrows():
    #print(index)
    if (index == 50000):
        print (50000)
        
    idx1 = int(row["Node1"])
    idx2 = int(row["Node2"])
    
    try :
        node1_values = w2vModel[str(idx1)]
    except KeyError:
        node1_values = [0.0 for i in range(nb_emb_w2v)]
    
    try :
        node2_values = w2vModel[str(idx2)]
    except KeyError:
        node2_values = [0.0 for i in range(nb_emb_w2v)]

    for i,(v1,v2) in enumerate(zip(node1_values, node2_values)):
        
        n1_emb_name = "1_Emb "+ str(i)
        n2_emb_name = "2_Emb "+ str(i)
        
        embDfTest.at[index, n1_emb_name] = v1
        embDfTest.at[index, n2_emb_name] = v2
        
pop1 = nlpTestDataframe.pop('Node1')
pop2 = nlpTestDataframe.pop('Node2')

/usr/local/Cellar/ipython/7.8.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:30: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/usr/local/Cellar/ipython/7.8.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:35: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


50000


In [9]:
predictions = model.predict_classes(nlpTestDataframe)
submissionDf = pd.DataFrame()
submissionDf["predicted"] = predictions.transpose()[0]
submissionDf.index.name = 'id'
file_name = "Results/predictionsTextNeural.csv"
submissionDf.to_csv(file_name, encoding='utf-8', index=True)

In [12]:
                                 # K FOLD TRAINING FOR GRADIENT BOOSTING
    
from sklearn import ensemble
    
n_estim = 100
n_split = 5
depth = 7

    
listModelBoosting = []

kf = KFold(n_splits=n_split, random_state=42, shuffle=True)# Define the split - into 10 folds 

val_scores = np.zeros((n_estim,), dtype=np.float64)

def heldout_score(clf, X_test, y_test):
    """compute deviance scores on ``X_test`` and ``y_test``. """
    score = np.zeros((n_estim,), dtype=np.float64)
    for i, y_pred in enumerate(clf.staged_decision_function(X_test)):
        score[i] = clf.loss_(y_test, y_pred)
    print(score)
    return score

for train_index, test_index in kf.split(embDf):
    
    X_train, X_test = nlpDataframe2.loc[train_index], nlpDataframe2.loc[test_index]
    y_train, y_test = Link[train_index], Link[test_index]

    clf = ensemble.GradientBoostingClassifier(learning_rate=0.1, n_estimators=n_estim, subsample=1.0,max_depth=depth)
    
    print('HERE')

    clf.fit(X_train, y_train)
    
    val_scores += heldout_score(clf, X_test, y_test)
    
    acc = clf.score(X_test, y_test)
    print("Accuracy: {:.4f}".format(acc))
    
    listModelBoosting.append(clf)

val_scores /= n_split

HERE
[1.29594696 1.27373839 1.25466698 1.23528288 1.21645553 1.20126694
 1.18467281 1.17308    1.16302512 1.15024002 1.14084685 1.13092677
 1.12415354 1.1171357  1.11001119 1.1024385  1.09626774 1.09101816
 1.08496277 1.0797193  1.07455534 1.07078925 1.06719471 1.06396056
 1.06035899 1.05823984 1.0504859  1.04798547 1.04573238 1.04296647
 1.04067904 1.03855144 1.03631589 1.03384419 1.02961174 1.02857625
 1.02637367 1.02508657 1.02321319 1.01827493 1.01670309 1.01467699
 1.01323254 1.01095614 1.00989913 1.00836825 1.00555194 1.00418636
 1.0022407  1.00116451 0.99977465 0.99846445 0.99767516 0.99620116
 0.99458607 0.99337245 0.99115587 0.98978069 0.98801811 0.9871
 0.98515446 0.9839949  0.98327291 0.9819938  0.98123801 0.98029994
 0.97789453 0.97581119 0.97523526 0.97463661 0.97292997 0.97092976
 0.97035535 0.96959147 0.96845785 0.96750184 0.96593014 0.9653037
 0.96418406 0.96333148 0.96269375 0.96089346 0.95941823 0.95770185
 0.95629156 0.9555987  0.9549648  0.95426589 0.95278357 0.9515

In [13]:
clf = listModelBoosting[3]

In [15]:
predictionsBoosting = clf.predict(nlpTestDataframe)
submissionDfBoosting = pd.DataFrame()
submissionDfBoosting["predicted"] = predictionsBoosting.transpose()
submissionDfBoosting.index.name = 'id'
file_name_boosting = "Results/predictionsTextBoosting.csv"
submissionDfBoosting.to_csv(file_name_boosting, encoding='utf-8', index=True)

In [22]:
                                 # K FOLD TRAINING FOR LOGISTIC REGRESSION
    
    
from sklearn.linear_model import LogisticRegression

n_split = 5
C_value = 1.0
loss = 'l2'
solv = 'lbfgs'
max_iter_value=200

listModelLogistic = []

kf = KFold(n_splits=n_split, random_state=42, shuffle=True)# Define the split - into 10 folds 


for train_index, test_index in kf.split(nlpDataframe2):
    
    X_train, X_test = nlpDataframe2.loc[train_index], nlpDataframe2.loc[test_index]
    y_train, y_test = Link[train_index], Link[test_index]

    clf = LogisticRegression(penalty = loss, C = C_value, solver = solv, max_iter = max_iter_value)
    
    print('HERE')

    clf.fit(X_train, y_train)
    
    acc = clf.score(X_test, y_test)
    
    print("Accuracy: {:.4f}".format(acc))
    
    listModelLogistic.append(clf)

HERE
Accuracy: 0.6345
HERE
Accuracy: 0.6356
HERE
Accuracy: 0.6366
HERE
Accuracy: 0.6344
HERE
Accuracy: 0.6342


In [23]:
clf = listModelLogistic[2]

In [24]:
predictionsLogistic = clf.predict(nlpTestDataframe)
submissionDfLogistic = pd.DataFrame()
submissionDfLogistic["predicted"] = predictionsLogistic.transpose()
submissionDfLogistic.index.name = 'id'
file_name_logistic = "Results/predictionsTextLogistic.csv"
submissionDfLogistic.to_csv(file_name_logistic, encoding='utf-8', index=True)